In [11]:
import undetected_chromedriver.v2 as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from importlib.machinery import SourceFileLoader
import time
from selenium import webdriver
import json

In [28]:
def get_driver(country_name='United States'):
    try:
        opt = uc.ChromeOptions()
        opt.add_argument("--no-sandbox")
        opt.add_argument("--disable-dev-shm-usage")
        opt.add_argument("--start-maximized")
        opt.add_argument("--disable-infobars")
        opt.add_argument('--disable-gpu')
#         opt.add_argument('--blink-settings=imagesEnabled=false')
        opt.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36")
        extension_folder = './chrome/extensions/'
        opt.add_argument('--load-extension='+extension_folder+'hoxx')
#         opt.add_argument('--load-extension='+extension_folder+'hoxx,'+extension_folder+'CSS-block,'+extension_folder+'GIF-blocker')
        driver = uc.Chrome(options = opt, use_subprocess=True)
        handles = driver.window_handles
        while len(handles) != 2:
            handles = driver.window_handles
        for idx,handle in enumerate(handles):
            if idx != 0:
                driver.switch_to.window(handle)
                driver.close()
        driver.switch_to.window(handles[0])
        driver.get('chrome-extension://nbcojefnccbanplpoffopkoepjmhgdgh/popup.html')
        language_flag = False
        loggedin_flag = False
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'mdc-list-item')))
        languages = driver.find_elements(By.CLASS_NAME, 'mdc-list-item')
        for language in languages:
            if language.text == 'English':
                language.click()
                language_flag = True
                break
            elif language.text == 'India':
                loggedin_flag = True
                break

        if language_flag:
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, 'email-input')))
            driver.find_element(By.ID, 'email-input').click()
            action = ActionChains(driver)
            action.send_keys('ttsst132@gmail.com')
            action.perform()
            driver.find_element(By.ID, 'password-input').click()
            action = ActionChains(driver)
            action.send_keys('bhargav19')
            action.perform()
            driver.find_element(By.ID, 'login-button').click()
            loggedin_flag = True

        if loggedin_flag:
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'mdc-list-item')))
            countries = driver.find_elements(By.CLASS_NAME, 'mdc-list-item')
            for country in countries:
                if country.text == country_name:
                    driver.execute_script("arguments[0].scrollIntoView();", country)
                    country.click()
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'connected-view__status-title')))
                    print('Created the final driver for Country: ' + country_name)
                    return driver
    except Exception as e:
        print(e)
    return None


In [60]:
driver = get_driver('United States')
driver.get('https://www.neimanmarcus.com/')

Created the final driver for Country: United States


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="silo-navigation"]"}
  (Session info: chrome=98.0.4758.102)
Stacktrace:
0   chromedriver                        0x00000001009bcee9 chromedriver + 5013225
1   chromedriver                        0x00000001009481d3 chromedriver + 4534739
2   chromedriver                        0x000000010051ea68 chromedriver + 170600
3   chromedriver                        0x0000000100553152 chromedriver + 385362
4   chromedriver                        0x0000000100553311 chromedriver + 385809
5   chromedriver                        0x0000000100585324 chromedriver + 590628
6   chromedriver                        0x000000010057080d chromedriver + 505869
7   chromedriver                        0x0000000100583074 chromedriver + 581748
8   chromedriver                        0x00000001005706d3 chromedriver + 505555
9   chromedriver                        0x000000010054676e chromedriver + 333678

In [62]:
nav = driver.find_element(By.ID, 'silo-navigation').find_element(By.TAG_NAME, 'ul')
departments = nav.find_elements(By.XPATH, './li')
departments_texts = []
for department in departments:
    departments_texts.append(department.find_element(By.CLASS_NAME, 'silo-link').text)
print(departments_texts)

['NEW & NOW', 'DESIGNERS', 'CLOTHING', 'SHOES', 'HANDBAGS', 'JEWELRY & ACCESSORIES', 'BEAUTY', 'KIDS', 'HOME', 'GIFTS', 'MAGAZINE', 'SALE']


In [80]:
conf = {}
conf['country'] = 'United States'
conf['retailer'] = 'Neimanmarcus'
conf['parsing_config'] = './retailer_config/neimanmarcuscom_parsing.json'
conf['gcloud_path'] = 'crawl_data/United States/Neimanmarcus/'
page_configs = []

In [81]:
for did, department in enumerate(departments):
#     if did != 6:
#         continue
    action = ActionChains(driver)
    action.move_to_element(department).perform()
    time.sleep(0.5)
    splits = department.find_elements(By.CLASS_NAME, 'menu-wrapper')
    categories = []
    for split in splits:
        categories += split.find_elements(By.XPATH, './div')
    
    for category in categories:
        category_landing_page = category.find_element(By.XPATH, './h6/a')
        page_config = conf.copy()            
        page_config['department'] = departments_texts[did].strip()
        page_config['category'] = category_landing_page.text.strip()
        page_config['page'] = category_landing_page.text.strip()
        page_config['page_url'] = category_landing_page.get_attribute('href')
        file_name = page_config['country'] + '_' + page_config['retailer'] + '_' + page_config['department'] + '_' + page_config['category'] + '_' + page_config['page']
        for k, v in json.load(open('config/encoding.json', 'r')).items():
            file_name = file_name.replace(k, v)
            file_name = file_name.replace('\\', '')
        page_config['file_name'] = file_name
        page_configs.append(page_config)
        
        pages = category.find_elements(By.XPATH, './ul/li/a')
        for page in pages:
            page_config = conf.copy()            
            page_config['department'] = departments_texts[did].strip()
            page_config['category'] = category_landing_page.text.strip()
            page_config['page'] = page.text.strip()
            page_config['page_url'] = page.get_attribute('href')
            file_name = page_config['country'] + '_' + page_config['retailer'] + '_' + page_config['department'] + '_' + page_config['category'] + '_' + page_config['page']
            for k, v in json.load(open('config/encoding.json', 'r')).items():
                file_name = file_name.replace(k, v)
                file_name = file_name.replace('\\', '')
            page_config['file_name'] = file_name
            page_configs.append(page_config)

        

In [83]:
with open("neimanmarcuscom_urls.json", "w") as outfile:
        json.dump(page_configs, outfile)

In [84]:
page_configs[0]

{'country': 'United States',
 'retailer': 'Neimanmarcus',
 'parsing_config': './retailer_config/neimanmarcuscom_parsing.json',
 'gcloud_path': 'crawl_data/United States/Neimanmarcus/',
 'department': 'NEW & NOW',
 'category': 'All New Arrivals',
 'page': 'All New Arrivals',
 'page_url': 'https://www.neimanmarcus.com/c/new-now-all-new-arrivals-cat84680809?navpath=cat000000_cat84680805_cat84680809',
 'file_name': 'United|sp|States_Neimanmarcus_NEW|sp||as||sp|NOW_All|sp|New|sp|Arrivals_All|sp|New|sp|Arrivals'}